In [341]:
import importlib.util
import os
import json
from dotenv import load_dotenv
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from simpleRegArchitecture import SimpleNet

load_dotenv() 

False

In [342]:
NUM_BINS = 5

config_file_path = os.getenv('CONFIG_FILE_PATH')
print(config_file_path)
with open(config_file_path, "r") as f:
    config = json.load(f)

#Load in the networks
output_dir = config.get("output_dir")
loaded_state_dict = torch.load(f'{output_dir}/trainedNetworks.pt')

#create new instantiations to contain the saved networks
num_networks = config.get("num_networks")
networks = []
for n in range(num_networks):
    networks.append(SimpleNet())

for i, network in enumerate(networks):
    state_dict_key = f'network_{i}'
    network.load_state_dict(loaded_state_dict[state_dict_key])




/Users/simoncole/seniorDesign/Large-Scale-Design-and-Analysis-of-Neural-Networks/config.json


In [343]:
layer_weight_distributions = []
layer_bin_ranges = []
network_fc_indices = []
#for each fully connected layer, create matrix of shape (N, M, B) where
#N = num neurons in layer
#M = num neurons in previous layer
#B = number of bins
#generate an array of bin edges based on min and max of the first network(for now)
layers = networks[0].children()
for index, layer in enumerate(layers):
    if isinstance(layer, nn.Linear):
        #for each layer make an array of shape (num of neurons in layer, num of inputs to layer, num of bins)
        network_fc_indices.append(index)
        layer_shape = layer.weight.shape
        layer_weight_distributions.append(np.zeros((NUM_BINS,) + layer_shape, dtype=int))
        #TODO: change to get the true min and max of all networks not just from the first network
        layer_bin_ranges.append(np.histogram_bin_edges(layer.weight.data.numpy().flatten(), bins=(NUM_BINS)))
        
#Generate for each layer a matrix of network, row, column matrix
network_weights = []
for index, layer_index in enumerate(network_fc_indices):
    network_weights_per_layer = []
    for network in networks:
        net_layers = list(network.children())
        network_weights_per_layer.append(net_layers[layer_index].weight.data.numpy())
    network_weights.append(network_weights_per_layer)

print(network_weights[0][0])

[[-0.20631538  0.73702973]
 [-0.47195265 -0.02429558]
 [ 0.07377665  0.01942262]
 [ 0.17985338 -0.24177411]
 [-0.56040883  0.49711204]
 [-0.7684532   0.01243034]
 [ 0.3990102   0.44647178]
 [-0.44980973 -0.07642356]
 [ 0.04446951 -0.5782668 ]
 [ 0.9490659  -0.40909216]]


In [344]:
#iterating over each layer
for layer_num, layer_distribution in enumerate(layer_weight_distributions):
    #iterate over all the weights in a layer and increment their corresponding index in the bin ranges to the 3D array 
    #select the first bin for iteration, could select any
    for i, row in enumerate(layer_distribution[0]):
        for j, weight_count in enumerate(row):
            #get the matching bin for the value of the weight
            #for this weight position, iterate over each network and select the layer you're already in
            #then get the bin for that weight and increment the corresponding bin and position
            for network in network_weights[layer_num]:
                #includes the left bin edge and excludes the right
                weight = network[i][j]
                corresponding_bin = np.digitize(weight, layer_bin_ranges[layer_num], right=False)
                #in the case the weight is > or < than the min max of the first network (will chnage)
                if weight < layer_bin_ranges[layer_num][0]:
                    corresponding_bin = 0
                elif weight > layer_bin_ranges[layer_num][len(layer_bin_ranges) - 1]:
                    corresponding_bin = len(layer_bin_ranges) - 2
                
                layer_weight_distributions[layer_num][corresponding_bin][i][j] += 1

print(layer_weight_distributions)

[array([[[5, 4],
        [4, 5],
        [5, 4],
        [4, 4],
        [4, 4],
        [3, 4],
        [4, 4],
        [2, 4],
        [4, 3],
        [4, 5]],

       [[0, 1],
        [1, 0],
        [0, 1],
        [1, 1],
        [1, 1],
        [2, 1],
        [1, 1],
        [3, 1],
        [1, 2],
        [1, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]]]), array([[[4, 5, 5, 5, 5, 4, 5, 5, 5, 4]],

       [[1, 0, 0, 0, 0, 1, 0, 0, 0, 1]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0,